# Example of using some simple tools for building a network model



First, let us load all the modules that will be needed.


In [ ]:
%matplotlib inline

from network import Network

from scipy.sparse import csr_matrix
from allensdk.config.model.formats.hdf5_util import Hdf5Util

import numpy as np
import pandas as pd

import csv

import matplotlib.pyplot as plt

import random
random.seed(5) # Use the seed to initialize random number generator.

## Define objects and functions that will be used later

Here is a simple way of defining connection probabilities for different cell types.

In [ ]:
# probabilities of establishing a connection from source -> target
connection_probabilities = { 
    ('excitatory', 'excitatory'): 0.5,
    ('excitatory', 'inhibitory'): 0.5,
    ('inhibitory', 'excitatory'): 0.6,
    ('inhibitory', 'inhibitory'): 0.8
}

Let us introduce a simple function for computing probability of connection between two cells based on the distance between their somata.

In [ ]:
def distance_probability(p1, p2): # p1 and p2 are 3D vectors.
    ''' convert euclidean distance to probability of connection '''
    dist = np.linalg.norm(p1 - p2, 2) # Compute the distance.
    
    dist_cutoff = 200.0
    if (dist >= dist_cutoff):
        return 0.0
    else:
        return ( 1.0 - dist / dist_cutoff ) ** 2.0 # Use a simple formula for probability depending on the distance.

Why don't we check how that looks like?

In [ ]:
# Keep one point fixed and sample one dimension in the other to plot the probability as a function of distance.
p1 = np.array([0.0, 0.0, 0.0])
p2 = np.array([1.0, 0.0, 0.0])
r_array = np.arange(0.0, 300.0, 2.5) # Array of distances.
p_array = np.array([]) # Array of probabilities.
for r in r_array:
    p = distance_probability(p1, p2 * r) # Use p2 * r to obtain [r, 0, 0]; the distance between that and [0,0,0] is r.
    p_array = np.append(p_array, p)
plt.plot(r_array, p_array)
plt.xlabel('Distance')
plt.ylabel('Probability of connection')
plt.show()

Now, let us define a function for assigning connections based on the probability function of our choice.

In [ ]:
def random_connectivity(src_i, tgt_i, source, target):
    ''' given a source target pair, create a connection (or not) '''
    pdist = distance_probability(positions[src_i],
                                 positions[tgt_i])

    # Take the distance-dependent probability and scale it by a factor
    # that depends on the target and source types.
    p = pdist * connection_probabilities[source['type'],target['type']]

    # Draw a random number for this pair of cells and decide whether the connection should be established or not.
    if random.random() < p:
        # Establish the connection!
        # Assign a number of synapses that the source cell makes on the target cell.
        return { 'nsyns': random.randrange(2, 6) }


Now, define a function that converts the object we use to store information about connections to a convenient matrix.

In [ ]:
def construct_matrix(connections):
    ''' turn a list of connection dictionaries into a numpy csr matrix '''
    cols = [ c['source'] for c in connections ]
    rows = [ c['target'] for c in connections ]
    data = [ c['nsyns'] for c in connections ]

    return csr_matrix((data, (rows, cols)))


## Populate the network

Generate a number of cells of different types; generate their positions.

In [ ]:
N_exc = 8
N_inh = 2
N = N_inh + N_exc

# cell positions
positions = 100.0 * np.random.random((N,3)) - 50.0 # x,y,z positions in a 100x100x100 box centered at (0,0,0).

# initialize the network
net = Network()

# add populations
net.add_population(N_exc, type='excitatory')
net.add_population(N_inh, type='inhibitory')

## Connect cells in the network

All it takes is to provide the function that we described above!

In [ ]:
# add the connectivity rule
net.connect(random_connectivity) 

# build the matrix
cells, connections = net.build()

print 'Total number of connections: ', len(connections)

## Save data to files and print out some information about the network

Get the information about cells.

In [ ]:
cells = pd.DataFrame.from_dict(cells)

cells['x'] = positions[:,0]
cells['y'] = positions[:,1]
cells['z'] = positions[:,2]

# Save the cells data to a csv file.
cells.to_csv('cells.csv', index=False)

print cells

Now, use the function we prepared above to convert the information about connections to a nice matrix.
Save that matrix to a binary HDF5 file.

In [ ]:
# turn it into a sparse matrix
if len(connections) > 0:
    m = construct_matrix(connections)
    print '(tgt, src)  nsyns'
    print m

    # write the connectivity to hdf5
    Hdf5Util().write('connections.h5', m)

